In [3]:
import sys
sys.path.append("..")
import math
import torch
import torch as th
from PIL import Image
import numpy as np

import torchvision as thv
# import torchmetrics as thm
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import pytorch_lightning as pl

from src.datamodule import BarcodeDM
from src.config import Config


/home/fatuus/deepschool-cvr-segment/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def is_dist_avail_and_initialized():
    if not th.distributed.is_available():
        return False
    if not th.distributed.is_initialized():
        return False
    return True

def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()

def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.inference_mode():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict

def train_one_epoch(model, optimizer, data_loader, device, epoch, scaler=None):
    model.train()
    loss_values = []
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)
        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )
    for i_batch, (images, targets) in enumerate(data_loader):
        # print(f"{i_batch=}")
        images = list(image.to(device) for image in images)
        targets = [
            {k: v.to(device) for k, v in t.items() if k!="ocr"} 
            for t in targets
        ]
        # with torch.cuda.amp.autocast(enabled=scaler is not None):
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        loss_value = losses_reduced.item()
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)
        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()
        if lr_scheduler is not None:
            lr_scheduler.step()
        loss_values.append(loss_value)
    return loss_values

In [5]:
cfg = Config.from_yaml("../config/baseline_detect.yml")
data = BarcodeDM(cfg.data_config, task=cfg.task, dry_run=True)

data.prepare_data()
data.setup()
train_loader = data.train_dataloader()


In [2]:
class DetectModel(pl.LightningModule):
    def __init__(self, cfg: Config):
        self.model = thv.models.detection.fasterrcnn_resnet50_fpn()
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
        # self.train_map = MeanAveragePrecision()
        self.val_map = MeanAveragePrecision()
        self.test_map = MeanAveragePrecision()
    
    def forward(self, x: torch.Tensor):
        return self.model(x)
    
    def configure_optimizers(self):
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(
            params, lr=0.05, momentum=0.9, weight_decay=0.0005
        )
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer, step_size=3, gamma=0.1
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]
            
    def training_step(self, batch, batch_idx):
        """
        """
        images, targets = batch
        loss_dict = self.model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        pred = self.model(images)
        self.val_map.update(
            preds=pred,target=targets
        )
        self.log("val_map", self.val_map, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        pred = self.model(images)
        self.test_map.update(
            preds=pred,target=targets
        )
        self.log("val_map", self.test_map, on_step=True, on_epoch=True)

    # def on_validation_epoch_start(self) -> None:
    #     pass

    #  def on_validation_epoch_end(self) -> None:
    #      # self.log_dict(self._val_cls_metrics.compute(), on_epoch=True, on_step=False)
    #      # self.log_dict(self._val_seg_metrics.compute(), on_epoch=True, on_step=False)
    #      pass

    # def on_test_epoch_end(self) -> None:
    #     # self.log_dict(self._test_cls_metrics.compute(), on_epoch=True, on_step=False)
    #     # self.log_dict(self._test_seg_metrics.compute(), on_epoch=True, on_step=False)
    #     pass

    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        self.optimizer.step()
        # self.lr_scheduler.step()  # Step per iteration


NameError: name 'pl' is not defined

In [1]:
model = DetectModel(config)

NameError: name 'DetectModel' is not defined

In [5]:
model = thv.models.detection.fasterrcnn_resnet50_fpn(
    # pretrained=True
    # weights="DEFAULT"
    
)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

In [7]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=0.05, momentum=0.9, weight_decay=0.0005
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=3, gamma=0.1
)

In [8]:
for epoch in range(5):
    losses = train_one_epoch(
        model, 
        optimizer, 
        train_loader, 
        device=th.device('cpu'), 
        epoch=0
    )
    lr_scheduler.step()
    print(th.mean(th.FloatTensor(losses)),":", losses)
    # def eval_step(model, data_loader):
    data_loader = data.train_dataloader()
    metric = MeanAveragePrecision()
    model.eval()
    for i_batch, (images, targets) in enumerate(data_loader):
        with th.no_grad():
            pred = model(images)
            losses = model(images, targets)
        # print(f"losses={losses}")
        metric.update(
            preds=pred,target=targets
        )
    print(f"metrics={metric.compute()}")
    print()

tensor(1.2451) : [1.3541381359100342, 1.3775279521942139, 1.0037707090377808]
metrics={'map': tensor(0.), 'map_50': tensor(0.), 'map_75': tensor(0.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.), 'mar_1': tensor(0.), 'mar_10': tensor(0.), 'mar_100': tensor(0.), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}

tensor(0.3949) : [0.5162265300750732, 0.5149989128112793, 0.15335889160633087]
metrics={'map': tensor(0.), 'map_50': tensor(0.), 'map_75': tensor(0.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.), 'mar_1': tensor(0.), 'mar_10': tensor(0.), 'mar_100': tensor(0.), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}

tensor(0.8522) : [0.41770943999290466, 1.842750906944275, 0.29606926441192627]
metrics={'map': tensor(0.), 'map_50': tensor(0

In [10]:
images

(tensor([[[0.3608, 0.3608, 0.3608,  ..., 0.5294, 0.5333, 0.6588],
          [0.3608, 0.3608, 0.3608,  ..., 0.5294, 0.5333, 0.6588],
          [0.3608, 0.3608, 0.3647,  ..., 0.5333, 0.5333, 0.6549],
          ...,
          [0.6039, 0.6039, 0.6039,  ..., 0.8941, 0.9059, 0.9176],
          [0.6039, 0.6039, 0.6039,  ..., 0.8941, 0.9059, 0.9176],
          [0.6078, 0.6039, 0.6078,  ..., 0.8941, 0.9059, 0.9176]],
 
         [[0.2745, 0.2745, 0.2745,  ..., 0.5020, 0.5059, 0.6314],
          [0.2745, 0.2745, 0.2745,  ..., 0.5020, 0.5059, 0.6314],
          [0.2745, 0.2745, 0.2784,  ..., 0.5059, 0.5059, 0.6275],
          ...,
          [0.5529, 0.5529, 0.5569,  ..., 0.9098, 0.9216, 0.9333],
          [0.5529, 0.5529, 0.5569,  ..., 0.9098, 0.9216, 0.9333],
          [0.5569, 0.5529, 0.5608,  ..., 0.9098, 0.9216, 0.9333]],
 
         [[0.2314, 0.2314, 0.2314,  ..., 0.4392, 0.4431, 0.5686],
          [0.2314, 0.2314, 0.2314,  ..., 0.4392, 0.4431, 0.5686],
          [0.2314, 0.2314, 0.2353,  ...,

In [12]:
targets

({'boxes': tensor([[592, 269, 839, 717]]),
  'ocr': '4605035006964',
  'labels': tensor([1])},
 {'boxes': tensor([[657, 173, 950, 762]]),
  'ocr': '4820240030508',
  'labels': tensor([1])})

In [11]:
pred[0]

{'boxes': tensor([[ 275.3355,  350.9869,  468.2901,  790.8179],
         [ 356.6255,  389.5027,  548.2433,  825.1721],
         [ 556.1147,  443.6824,  747.8806,  868.8380],
         [ 194.1274,  346.7745,  386.8185,  784.0795],
         [ 635.1625,  398.5859,  827.3735,  822.6014],
         [ 476.9675,  396.8798,  670.3206,  828.6777],
         [ 596.6187,  522.2919,  788.1330,  943.2504],
         [ 402.0470,  405.9496,  981.7090,  871.9981],
         [ 515.6664,  522.2070,  707.3858,  947.6672],
         [ 236.3058,  442.1051,  426.7404,  870.9706],
         [ 593.3947,  318.1206,  787.8076,  744.8722],
         [ 195.7943,  600.8182,  387.4885, 1031.0927],
         [ 433.3954,  649.4655,  731.2982,  970.6425],
         [ 197.3358,  169.4981,  388.4341,  590.7922],
         [ 316.5822,  561.7359,  508.3601,  984.6205],
         [ 717.3361,  427.3879,  909.5702,  860.5986],
         [ 191.5795,  335.3460,  490.1313,  659.9505],
         [ 153.1497,  442.9861,  345.5234,  871.3863],
 

In [18]:
type(model)
#(images,targets)

torchvision.models.detection.faster_rcnn.FasterRCNN

In [ ]:
[x for x in pred if x["scores"].nelement()]

In [19]:
thv.models.detection.faster_rcnn.__file__

'/home/fatuus/deepschool-cvr-segment/venv/lib/python3.10/site-packages/torchvision/models/detection/faster_rcnn.py'

In [ ]:
metric.compute()

In [ ]:
thv.ops.box_iou?

In [ ]:
targets

In [ ]:
model(images, targets)

In [ ]:
_=model.eval()

In [ ]:
model(images)

In [ ]:
thv.ops.box_iou(
    [x for x in pred if x["scores"].nelement()],
    targets
)

In [ ]:
b.keys()

In [ ]:
b["label"]

In [ ]:
b["boxes"] = b["label"]

In [ ]:
import torch as th

In [ ]:
th.stack(b["boxes"], axis=1)

In [ ]:
b["boxes"]

In [ ]:
targets = []
for bbox in b["label"]:
    targets.append(
        {
            "boxes": 
        }
    )

In [ ]:
th2pil = thv.transforms.ToPILImage()

In [ ]:
th2pil(b["image"][0].tra)

In [ ]:
modelb["image"][:1], [{"boxes":th.stack(b["boxes"], axis=1)}])